## Introduction

THIS NOTEBOOK IS A WORK IN PROGRESS! In this notebook I plan to do some exploratory data analysis and create some simple linear models. I might add tree-based models at some point.

## Data Preparation

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import seaborn as sns
import warnings

# Do not limit number of columns in output.
pd.set_option('display.max_columns', None)

warnings.filterwarnings('ignore')

['sample_submission.csv', 'test.csv', 'train.csv']


In [10]:
train_data = pd.read_csv('../input/train.csv')

Let's take a peek on the data to see how it looks like:

In [11]:
train_data.head()

,id,belongs_to_collection,budget,genres,homepage,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,runtime,spoken_languages,status,tagline,title,Keywords,cast,crew,revenue
0,1,"[{'id': 313576, 'name': 'Hot Tub Time Machine ...",14000000,"[{'id': 35, 'name': 'Comedy'}]",NaN,tt2637294,en,Hot Tub Time Machine 2,"When Lou, who has become the ""father of the In...",6.575393,/tQtWuwvMf0hCc2QR2tkolwl7c3c.jpg,"[{'name': 'Paramount Pictures', 'id': 4}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",2/20/15,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The Laws of Space and Time are About to be Vio...,Hot Tub Time Machine 2,"[{'id': 4379, 'name': 'time travel'}, {'id': 9...","[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",12314651
1,2,"[{'id': 107674, 'name': 'The Princess Diaries ...",40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,tt0368933,en,The Princess Diaries 2: Royal Engagement,Mia Thermopolis is now a college graduate and ...,8.248895,/w9Z7A0GHEhIp7etpj0vyKOeU1Wx.jpg,"[{'name': 'Walt Disney Pictures', 'id': 2}]","[{'iso_3166_1': 'US', 'name': 'United States o...",8/6/04,113.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,It can take a lifetime to find true love; she'...,The Princess Diaries 2: Royal Engagement,"[{'id': 2505, 'name': 'coronation'}, {'id': 42...","[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",95149435
2,3,NaN,3300000,"[{'id': 18, 'name': 'Drama'}]",http://sonyclassics.com/whiplash/,tt2582802,en,Whiplash,"Under the direction of a ruthless instructor, ...",64.299990,/lIv1QinFqz4dlp5U4lQ6HaiskOZ.jpg,"[{'name': 'Bold Films', 'id': 2266}, {'name': ...","[{'iso_3166_1': 'US', 'name': 'United States o...",10/10/14,105.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The road to greatness can take you to the edge.,Whiplash,"[{'id': 1416, 'name': 'jazz'}, {'id': 1523, 'n...","[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",13092000
3,4,NaN,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",http://kahaanithefilm.com/,tt1821480,hi,Kahaani,Vidya Bagchi (Vidya Balan) arrives in Kolkata ...,3.174936,/aTXRaPrWSinhcmCrcfJK17urp3F.jpg,NaN,"[{'iso_3166_1': 'IN', 'name': 'India'}]",3/9/12,122.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,NaN,Kahaani,"[{'id': 10092, 'name': 'mystery'}, {'id': 1054...","[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",16000000
4,5,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,tt1380152,ko,마린보이,Marine Boy is the story of a former national s...,1.148070,/m22s7zvkVFDU9ir56PiiqIEWFdT.jpg,NaN,"[{'iso_3166_1': 'KR', 'name': 'South Korea'}]",2/5/09,118.0,"[{'iso_639_1': 'ko', 'name': '한국어/조선말'}]",Released,NaN,Marine Boy,NaN,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",3923970


Some columns have data in the format of list of objects. These cannot be directly used as input for learning algorithm so we convert them to CSV (comma-separated values) by taking the attribute that gives us more information and discard the rest. For example, for `genres` column we project the `name` attribute of the object and discard `id`s.

In [ ]:
import ast

def flatten_field(field, attribute):
    """
    Convert a field from Python AST representation to a plain CSV string. To do that, project
    only `attribute` and ignore remaining field of the object.
    
    For example, the original field
        {'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Action'}
    is converted to
        "comedy,action"
        
    Note that this function converts all values to lower case.
    """
    if pd.isna(field):
        return ''
    else:
        obj_list = ast.literal_eval(field)
        result = []
        for obj in obj_list:
            result.append(obj[attribute].lower())
            
    return ','.join(result)


def flatten_data(data):
    col_attribute_mapping = {'belongs_to_collection': 'name', 
                             'genres': 'name', 
                             'production_countries': 'iso_3166_1',
                             'production_companies': 'name',
                             # For spoken language we actually want the ISO code instead of name to 
                             # avoid non-ASCII characters.
                             'spoken_languages': 'iso_639_1',
                             'Keywords': 'name', 
                             'cast': 'name', 
                             'crew': 'name'}
    
    for col in col_attribute_mapping.keys():
        data[col] = data.apply(lambda row: flatten_field(row[col], col_attribute_mapping[col]), 
                                           axis=1)

Now actually calling the function to flatten data.

In [ ]:
flatten_data(train_data)
train_data.head()

Run the code cell below to generate a CSV file with the flatten data.

In [ ]:
train_data.to_csv('train_flatten.csv', index=False)

## Exploratory Data Analysis

We can start thinking about variable selection, that is, what columns do we think that can provide us information predicting the revenue. Some variables are obvious candidates: barring some big flops, films with bigger budgets are better produced and consequently interest more people. `popularity` is another good one, as it indirectly tells how many people have watched, or are at least familiar with the film. There are other less obvious columns that can still provide good information, like genres column, as some genres are more popular than others. This is also the case for `spoken_languages` column, as it tells us what is the main market for the movie.
Let's now investigate the distribution of `revenue` variable.

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))
fig.suptitle('Revenue Distribution', fontsize=15)
sns.distplot(train_data['revenue'], bins=50, kde=False)
ax.grid()

We can see the distribution of revenue is right skewed, and this is a problem for linear regression. Later on we will see how to transform the target variable for a better fit.

Some plots to give us an idea of how some of the variables relate to the target variable, `revenue`.

_Budget vs Revenue_

In [ ]:
sns.jointplot(x="budget", y="revenue", data=train_data, height=11, ratio=4, color="g")
plt.show()

_Popularity vs Revenue_

In [ ]:
sns.jointplot(x="popularity", y="revenue", data=train_data, height=11, ratio=4, color="g")
plt.show()

_Year vs Average Revenue_

In [ ]:
# Copied from https://www.kaggle.com/kamalchhirang/eda-feature-engineering-lgb-xgb-cat.
#Since only last two digits of year are provided, this is the correct way of getting the year.
train_data[['release_month','release_day','release_year']] = train_data['release_date'].str.split('/',expand=True).replace(np.nan, -1).astype(int)
# Some rows have 4 digits of year instead of 2, that's why I am applying (train['release_year'] < 100) this condition
train_data.loc[ (train_data['release_year'] <= 19) & (train_data['release_year'] < 100), "release_year"] += 2000
train_data.loc[ (train_data['release_year'] > 19)  & (train_data['release_year'] < 100), "release_year"] += 1900

In [ ]:
train_data.head()

In [ ]:
avg_revenue_by_year = train_data.groupby(['release_year'], as_index=False)['revenue'].mean()

sns.jointplot(x='release_year', y='revenue', data=avg_revenue_by_year, height=11, ratio=4, color="g")
plt.show()

_Original Language vs Average Revenue_

In [ ]:
avg_revenue_by_language = train_data.groupby(['original_language'], as_index=False)['revenue'].mean()
plt.figure(figsize=(16, 6))
sns.barplot(x='original_language', y='revenue', data=avg_revenue_by_language)
plt.show()

## A Simple Linear Regression Model

Our first attempt is to create a simple linear model including only numerical variables:

In [ ]:
train_data.describe()

In [ ]:
# Note we are not including 'id' column.
train_data_subset = train_data[['budget', 'popularity', 'revenue']]

Before creating a simple model using Linear Regression we need to make sure there are no NA values in the dataset:

In [ ]:
np.sum(pd.isna(train_data_subset))

We see there are two rows with NA in the runtime column so we removem them before fitting the linear regression model:

In [ ]:
train_data_subset = train_data_subset.fillna(0)

np.sum(pd.isna(train_data_subset))

We now procceed to fit and evaluate a simple linear regression model. The library `statsmodels` offers many statistical methods, including linear regression. As these offer out-of-the-box model summaries we use them. Later on we will use the version present in `scikit-learn` library.

In [ ]:
from scipy import stats

revenue = stats.boxcox(train_data_subset['revenue'])[0]

revenue.head()


In [ ]:
import statsmodels
import statsmodels.formula.api as smf


lr_model = smf.ols('revenue ~ budget + popularity', data=train_data_subset).fit()

In [ ]:
lr_model.summary()

Some observations about the model summary above:
* F-statistic is 1593, way higher than 1. This offers compelling evidence that the null hypothesis - that all coefficients are zero - is not true.
* After some assurance from F-statistic, we look into `t` values for each coefficient. Again, we have high values and they are all significant $P>|t| < 0.05$.
* The $R^2$ of around 0.61 shows the model fits well the data.

Taking `budget` coefficient as an example, we see that a 100,000 dollars increase in the budget is associated with an additional 248,230 dollars in revenue.


### Assessing the Accuracy of the Model

#### Residual Standard Error

The residual standard error is defined as

$$RSE=\sqrt{\frac{1}{n-p-1}RSS}$$

where

$$RSS=\sum_{i=1}^{n}(y_i-\hat{y_i})^2$$

$n$ is the number of observations and $p$ is the number of predictors.

In [ ]:
# Number of observations, number of columns
n, n_cols = train_data_subset.shape
# Number of predictors
p = n_cols - 1
rss = np.sum(np.power(lr_model.resid, 2))
rse = np.sqrt((1/(n-p-1)) * rss)
rse

$RSE$ is an estimate of the standard deviation of the error. Roughly speaking,  it is the average amount that the response will deviate from the true regression line. In our example, it means that the predicted value will deviate from true value by approximately $85424537, on average. To have a better idea of the impact of this absolute number we compare it to the average revenue:

In [ ]:
rse / np.mean(train_data_subset['revenue'])

There's conflicting information: the high $R^2$ value indicates there's a good fit but the high $RSE$ shows the opposite, that the predicted values deviate from the true model. In the next session we will see how we can transform our data to get better results.

### Checking Linear Regression Assumptions

When we fit a linear regression to data we assume the following holds:
1. The relationship between the response variable $y$ and the predictors is approximately linear.
2. The error term $\epsilon$ has zero mean.
3. The error term $\epsilon$ has constant variance $\sigma^2$.
4. The errors are uncorrelated.
5. The errors are normally distributed.

Taken together, assumptions 4 and 5 imply that the errors are independent random variables.

#### Linearity of Predictors-Response Relationship

We can use residual plots to identify non-linearity.

In [ ]:
#fig, ax = plt.subplots(figsize=(6,2.5))
#ax.scatter(lr_model.fittedvalues, lr_model.resid)
import seaborn as sns

fit_data = pd.DataFrame({'Fitted': lr_model.fittedvalues, 'Residuals': lr_model.resid})

sns.regplot(x='Fitted', y='Residuals', data=fit_data)

There's no much pattern in the residuals but there's a strong concentration of points in the lower left corner. This could indicate a non-linearity in the data.

#### Normality

In [ ]:
import scipy as sp
fig, ax = plt.subplots(figsize=(6,2.5))
_, (__, ___, r) = sp.stats.probplot(lr_model.resid, plot=ax, fit=True)
r**2

## Predictive Performance

Now we test the predictive performance of the linear model on a hold-out test set. To do that we take the original training set and split that into a new training set and a test set. We then train the model on the new training set, and predict values for the test set. As we have the labels for the test set we created we are able to measure the performance.

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split

y = train_data['revenue']
X = train_data[['budget', 'popularity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lm = LinearRegression().fit(X_train, y_train)
y_predict = lm.predict(X_test)
y_predict[y_predict < 0] = 0
print("Mean squared error (linear model): {:.2f}".format(mean_squared_error(y_test, y_predict)))
print("Mean squared log error (linear model): {:.2f}".format(mean_squared_log_error(y_test, y_predict)))
print("r2_score (linear model): {:.2f}".format(r2_score(y_test, y_predict)))

Scaling data:

In [ ]:
from sklearn.preprocessing import StandardScaler

y = train_data['revenue']
X = train_data[['budget', 'popularity']]

X.head()

X.loc[:,['budget', 'popularity']] = StandardScaler().fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lm = LinearRegression().fit(X_train, y_train)
y_predict = lm.predict(X_test)
y_predict[y_predict < 0] = 0
print("Mean squared error (linear model): {:.2f}".format(mean_squared_error(y_test, y_predict)))
print("Mean squared log error (linear model): {:.2f}".format(mean_squared_log_error(y_test, y_predict)))
print("r2_score (linear model): {:.2f}".format(r2_score(y_test, y_predict)))

In [ ]:
y = np.log(train_data['revenue'] + 1)
X = train_data[['budget', 'popularity']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
lm_log = LinearRegression().fit(X_train, y_train)
y_predict = lm_log.predict(X_test)
y_predict[y_predict < 0] = 0
print("Mean squared error (linear model, log(revenue)): {:.2f}".format(mean_squared_error(y_test, y_predict)))
print("Mean squared log error (linear model, log(revenue)): {:.2f}".format(mean_squared_log_error(y_test, y_predict)))
print("r2_score (linear model(log(revenue))): {:.2f}".format(r2_score(y_test, y_predict)))

In [ ]:
# Preparing submission from linear model and linear model on log-transformed data.

test_data = pd.read_csv('../input/test.csv')
ids_test = test_data['id']
test_data_subset = test_data[['budget', 'popularity']]

y = train_data['revenue']
X = train_data[['budget', 'popularity']]

mean_y = np.mean(y)

lm = LinearRegression().fit(X, y)
y_predict = lm.predict(test_data_subset)

pred_df = pd.DataFrame({'id': ids_test, 'revenue': y_predict})
pred_df.loc[pred_df.revenue < 0, ['revenue']] = mean_y
pred_df.to_csv('lm_predictions.csv', index=False)

y = np.log(train_data['revenue'] + 1)
X = train_data[['budget', 'popularity']]

lm_log = LinearRegression().fit(X, y)
y_predict = lm_log.predict(test_data_subset)

pred_df = pd.DataFrame({'id': ids_test, 'revenue': np.exp(y_predict) - 1})
pred_df.loc[pred_df.revenue < 0, ['revenue']] = mean_y
pred_df.to_csv('lm_predictions_log.csv', index=False)
pred_df.head()

In [ ]:
mean_y

### Adding Qualitative Predictors

So far we only included numerical predictors into our model. We can also include qualitative predictors into our model provided we convert them to a suitable representation.

It makes sense to assume that the language of the movie impacts revenue, as it is a proxy to the country it was filmed and consequently size of the market.

## Tree-based Methods

We procceed to fit XGBoost to the data to check how it performs. First, let's load again the data and preprocess it.

In [ ]:
train_data = pd.read_csv('../input/train.csv')
flatten_data(train_data)

In [ ]:
train_data.head()

Create the dataset that will be using for training our model. We will take some columns as is, dummy encode others and discard the rest.

Some columns have a high number of unique values and adding dummies for all values may not help the model. One way of going about it is to select the top $N$ most frequent values for the field and encode based on those. Instead, we dummify the field and remove resulting columns that have low number of 1 values, which has the same effect.

In [ ]:
def count_unique(data, field_name):
    s = set()
    
    def count(field):
        for genre in field.split(','):
            s.add(genre)
        
    data.apply(lambda x: count(x[field_name]), axis=1)
    return s

print("Distinct values for 'genres':", len(count_unique(train_data, 'genres')))
print("Distinct values for 'original_language':", len(count_unique(train_data, 'original_language')))
print("Distinct values for 'production_companies':", len(count_unique(train_data, 'production_companies')))
print("Distinct values for 'production_countries':", len(count_unique(train_data, 'production_countries')))
print("Distinct values for 'spoken_languages':", len(count_unique(train_data, 'spoken_languages')))
print("Distinct values for 'Keywords':", len(count_unique(train_data, 'Keywords')))
print("Distinct values for 'cast':", len(count_unique(train_data, 'cast')))

In [ ]:
def is_high_variance(data):
    return (np.sum(data) / data.shape[0]) > 0.15


def dummify_columns(data, add_label=True):
    # production_companies, Keywords and cast columns will be dealt with later.
    columns_to_dummify = ['genres', 'original_language', 'production_countries', 'spoken_languages']
    processed_data = data

    for column in columns_to_dummify:
        # Split column in values separated by comma and create dummies based on that instead
        # of considering the column value as a whole.
        dummies = processed_data[column].str.get_dummies(sep=",")
        processed_data.drop([column], axis=1, inplace=True)
        # Prefix dummy columns with the name of the original column so we don't end up with duplicated
        # column names when columns share the same level (what breaks XGBoost).
        dummies.columns = [column + "_" + dummy_column for dummy_column in dummies.columns ]
        before_len = dummies.shape[1]
        # Get rid of columns that do not provide much information.
        dummies = dummies.loc[:, is_high_variance(dummies)]
        after_len = dummies.shape[1]
        print(column, before_len, ' -> ', after_len)
        processed_data = pd.concat([processed_data, dummies], axis=1)
        
    return processed_data

In [ ]:
processed_train_data = dummify_columns(train_data)
processed_train_data.head()

Split `release_date` into separate fields and use only month and year.

In [ ]:
from datetime import datetime

def add_delta_col(row):
    format = "%m/%d/%Y"
    first_day_str = '1/1/' + str(row['release_year'])
    first_date = datetime.strptime(first_day_str, format)
    release_date_str = str(row['release_month']) + '/' + str(row['release_day']) + '/' + str(row['release_year'])
    release_date = datetime.strptime(release_date_str, format)
    
    return (release_date - first_date).days
    

def add_date_features(data):
    date_format = "%m/%d/%Y"
    
    # Split 'release_date' into separate fields for month and year, ignoring day.
    date_features = data['release_date'].str.split('/', expand=True).replace(np.nan, 1).astype(int)
    data['release_month'] = date_features[0]
    data['release_day'] = date_features[1]
    data['release_year'] = date_features[2]
    data.loc[(data['release_year'] <= 19) & (data['release_year'] < 100), "release_year"] += 2000
    data.loc[(data['release_year'] > 19)  & (data['release_year'] < 100), "release_year"] += 1900
    data['release_date_delta'] = data.apply(add_delta_col, axis=1)

In [ ]:
add_date_features(processed_train_data)
processed_train_data.head()

Testing the performance of the system on a hold-out testset.

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_squared_log_error, r2_score
from sklearn.model_selection import train_test_split


def report_scores(y, y_hat):
    print("Mean squared error (XGBoost): {:.2f}".format(mean_squared_error(y_test, y_predict)))
    print("Mean squared log error (XGBoost): {:.2f}".format(mean_squared_log_error(y_test, y_predict)))
    print("r2_score (XGBoost): {:.2f}".format(r2_score(y_test, y_predict)))    

    
y = processed_train_data['revenue']
X = processed_train_data.drop(['revenue'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
xgb_model = XGBRegressor(objective="reg:linear", random_state=0).fit(X_train, y_train)
y_predict = xgb_model.predict(X_test)
y_predict[y_predict < 0] = 0
report_scores(y_test, y_predict)


Preparing data to find best parameters using grid search.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, make_scorer

train_data = pd.read_csv('../input/train.csv')
flatten_data(train_data)
add_date_features(train_data)
columns_to_drop = ['id', 'belongs_to_collection', 'homepage', 'imdb_id', 'original_title', 'overview',
                   'poster_path', 'status', 'tagline', 'title', 'Keywords', 'cast', 'crew',
                  'production_companies', 'release_date', 'release_day']

train_data = train_data.drop(columns_to_drop, axis=1)
train_data = dummify_columns(train_data)

train_data.head()

In [ ]:
#scorer = make_scorer(mean_squared_error, greater_is_better=False)
from scipy import stats, special

grid_values = {
    'colsample_bytree': [0.5],
    'subsample': [0.9],
    'max_depth': [3, 5, 7],
    'n_estimators': [300, 500, 600, 700, 800],
    'gamma': [0],
    'min_child_weight': [2, 3, 4, 5],
    'learning_rate': [0.01, 0.02, 0.03, 0.04, 0.05]
}

y = np.log(train_data['revenue'] + 1)
X = train_data.drop(['revenue'], axis=1)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# UNCOMMENT NEXT LINE TO PERFORM GRID SEARCH.
#xgb_model = XGBRegressor(objective="reg:linear", n_jobs=4, random_state=0)
# Best so far.
# xgb_model = XGBRegressor(objective="reg:linear", n_jobs=4, learning_rate=0.05, n_estimators=300, max_depth=3, min_child_weight=5, random_state=0)
# Experimenting with tuning colsample_bytree and subsample first.
#grid_clf_acc = GridSearchCV(xgb_model, grid_values, fit_params={'eval_metric': 'rmse'}, scoring=scorer)
grid_clf_acc = GridSearchCV(xgb_model, grid_values, n_jobs=4, cv=5, fit_params={'eval_metric': 'rmse'}, scoring='neg_mean_squared_error')
grid_clf_acc.fit(X_train, y_train)
#y_decision_fn_scores_acc = grid_clf_acc.decision_function(X_test) 

print('Grid best parameter (max. MSE): ', grid_clf_acc.best_params_)
print('Grid best score (MSE): ', grid_clf_acc.best_score_)

Visualizing model feature importances:

In [ ]:
xgb.plot_importance(grid_clf_acc.best_estimator_)

Creating a submission for XGBoost.

In [ ]:
train_data = pd.read_csv('../input/train.csv')
test_data = pd.read_csv('../input/test.csv')
len_train_data = train_data.shape[0]
y = np.log(train_data['revenue'] + 1)
train_data = train_data.drop(['revenue'], axis=1)
ids_test = test_data['id']

# Create a single dataset to do preprocessing and split for training and prediction.
data = pd.concat([train_data, test_data], axis=0)
flatten_data(data)
add_date_features(data)
columns_to_drop = ['id', 'belongs_to_collection', 'homepage', 'imdb_id', 'original_title', 'overview',
                   'poster_path', 'status', 'tagline', 'title', 'Keywords', 'cast', 'crew',
                  'production_companies', 'release_date', 'release_day']
data = data.drop(columns_to_drop, axis=1)
data = dummify_columns(data)

In [ ]:
data.head()

In [ ]:
import xgboost as xgb

X_train = data[0:len_train_data]
X_test = data[len_train_data:data.shape[0]]
xgb_model = XGBRegressor(objective="reg:linear", n_jobs=4, learning_rate=0.03, n_estimators=500, max_depth=3, min_child_weight=5, colsample_bytree=0.5, subsample=0.9, random_state=0)
xgb_model.fit(X_train, y, eval_metric='rmse')

y_predict = np.exp(xgb_model.predict(X_test)) - 1

pred_df = pd.DataFrame({'id': ids_test, 'revenue': y_predict})
pred_df.loc[pred_df.revenue < 0, ['revenue']] = 0
pred_df.to_csv('xgb_predictions.csv', index=False)
pred_df.head()

### PCA

Principal Component Analysis (PCA) transforms data by finding linear combination of predictors. The first PC captures most of the variance of the data and subsequent PCs capture increasing portions of the variance of the data. Can be used on a group of highly correlated features as an alternative of manually creating a linear combination of the features or choosing arbitrarily a subset of features. PCA creates uncorrelated components, what helps some predictive methods.

PCA seeks linear combinations of features that maximize variability and thus if the measurement scales of the original predictors differ in orders of magnitude, the first PC will focus on summarizing higher magnitude predictors (e.g. income in dollars vs height in centimeters). Important to transform skewed predictors and then center and scale the data before applying PCA.

(Kjell and Kuhn, Applied Predictive Modeling)

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

X_normalized = StandardScaler().fit(train_data).transform(train_data)

pca = PCA(n_components=2).fit(X_normalized)

X_pca = pca.transform(X_normalized)



In [ ]:
col = X_train['release_month'].astype(str)

### Feature Ideas to Explore

- number of days since the beginning of the year to capture seasonal trends
- adjust budget by inflation
- SVD on Keywords
- check for outliers
- transformations on numerical predictors?
- TF-IDF and Non-negative Matrix Factorization for keywords